# Olist E-commerce Performance & Profitability Analysis  
## Notebook 01: Data Cleaning & Integration

### Objective
This notebook focuses on the **data understanding, inspection, cleaning, and integration** of all Olist datasets.  
The goal is to create one **clean, consistent, and analysis-ready master table** that combines data from all sources.

### Business Context
Olist is a Brazilian e-commerce marketplace connecting small businesses (sellers) to customers nationwide.  
However, the company has faced **declining customer satisfaction** and **inconsistent profitability** across categories.

To help the business recover, I aim to:
1. Identify the **key causes of customer dissatisfaction** (e.g., delays, product issues).
2. Find the **drivers of low profitability** (e.g., high shipping cost, poor-performing sellers).
3. Prepare clean data for **visual and dashboard analysis** in the next phase.

### This Notebook Covers:
1. Data Loading — importing all 9 CSV files  
2. Initial Inspection — overview of structure, size, and missing values  
3. Data Cleaning — handling missing values, date formatting, category mapping  
4. Data Integration — merging multiple tables into one “master” dataset  
5. Export — saving the cleaned dataset for the next analysis notebook

## Step 1: Import Required Libraries
I’ll import all the libraries required for data loading, cleaning, and merging.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Dataset Overview: Olist Data Dictionary

| File Name | Primary Purpose | Foreign Keys (for joining) |
| :--- | :--- | :--- |
| **`olist_orders_dataset.csv`** | **Master Order Table** (Status, Dates, Customer ID) | `customer_id` |
| **`olist_customers_dataset.csv`** | Customer location and unique user ID. | `customer_id` (from Orders) |
| **`olist_geolocation_dataset.csv`** | Geographical coordinates for zip codes. | `geolocation_zip_code_prefix` (from Customers/Sellers) |
| **`olist_order_items_dataset.csv`** | Transactional core (price, seller, product). | `order_id`, `product_id`, `seller_id` |
| **`olist_order_payments_dataset.csv`** | Payment method, installments, and total value. | `order_id` |
| **`olist_order_reviews_dataset.csv`** | Customer satisfaction score (1-5) and message. | `order_id` |
| **`olist_products_dataset.csv`** | Physical details, attributes, and category name. | `product_id` (from Order Items) |
| **`olist_sellers_dataset.csv`** | Seller registration and location information. | `seller_id` (from Order Items) |
| **`product_category_name_translation.csv`** | Look-up table to convert Portuguese names to English. | `product_category_name` (from Products) |

---

### Loading all files

In [20]:
orders_data = pd.read_csv('../raw_data/olist_orders_dataset.csv')
customers_data = pd.read_csv('../raw_data/olist_customers_dataset.csv')
geolocation_data = pd.read_csv('../raw_data/olist_geolocation_dataset.csv')
orders_item_data = pd.read_csv('../raw_data/olist_order_items_dataset.csv')
orders_payment_data = pd.read_csv('../raw_data/olist_order_payments_dataset.csv')
orders_review_data = pd.read_csv('../raw_data/olist_order_reviews_dataset.csv')
products_data = pd.read_csv('../raw_data/olist_products_dataset.csv')
sellers_data = pd.read_csv('../raw_data/olist_sellers_dataset.csv')
category_names_data = pd.read_csv('../raw_data/product_category_name_translation.csv')

In [21]:
def inspect(df, df_name='df'):
    print(df_name)
    print(f"\nShape: {df.shape}")
    print("\nDtypes & Null Counts:")

    df.info()
    print(f"\nData Preview: {df.head()}")
    print(f"\nNull Count: {df.isnull().sum()}\n")

In [22]:
data_objects_map = {
    'Orders Data': orders_data,
    'Customers Data': customers_data,
    'GeoLocation Data': geolocation_data,
    'Orders Items Data': orders_item_data,
    'Orders Payments Data': orders_payment_data,
    'Orders Reviews Data': orders_review_data,
    'Products Data': products_data,
    'Sellers Data': sellers_data,
    'Category Names Data': category_names_data
}

for name, data in data_objects_map.items():
    inspect(data, name)

Orders Data

Shape: (99441, 8)

Dtypes & Null Counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB

Data Preview:                            order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb474

## Initial Data Inspection Summary

### olist_orders_dataset

This is the main **orders timeline** table ($\approx 99.4k$ records). All date columns are currently stored as `object` (string) and **must be converted** to datetime objects for analysis. Approximately **3% of orders lack a customer delivery date** and $\approx 1.8\%$ lack a carrier dispatch date, strongly indicating **canceled or undelivered orders**.

### olist_customers_dataset

This dataset is clean and complete ($\approx 99.4k$ records with 0 nulls). The critical feature is the **`customer_unique_id`**, which is necessary for distinguishing between one-time buyers and repeat customers for RFM analysis.

### olist_geolocation_dataset

This table is **very large** ($\approx 1$ million records) and fully complete. Given its size, I must confirm that I **only load the required zip codes** during the merge phase to avoid memory issues, even after optimizing its high memory usage (38+ MB).

### olist_order_items_dataset

This transactional table is complete and contains **$\approx 112.6k$ item records** linked to $\approx 99.4k$ orders. Since it contains multiple entries per `order_id` (for multiple items), I must be careful to aggregate prices and freight values correctly when merging.

### olist_order_payments_dataset

This table has $\approx 103.8k$ records, slightly more than the number of orders, confirming that some **orders were paid for using multiple payment types** (e.g., credit card and voucher) or sequential transactions. No critical missing values were observed.

### olist_order_reviews_dataset

This table has major sparsity issues in its qualitative data. While review scores are complete, **$\approx 88\%$ of reviews lack a comment title** and $\approx 59\%$ lack a message. I can only rely on the **`review_score`** for satisfaction analysis unless advanced text analysis is performed on the available messages.

### olist_products_dataset

This catalog of $\approx 32.9k$ unique products is relatively clean. The main issue is that $\approx 2\%$ of records lack product attribute data (category, length, weight), which means these **products cannot be categorized or used in logistics analysis** (freight cost estimation).

### olist_sellers_dataset

This small table ($\approx 3k$ records) is fully complete and will be critical for linking seller performance metrics (derived from **Order Items**) to their geographical location.

### product_category_name_translation.csv

This small, complete table ($\approx 71$ records) is essential for converting the Portuguese category names into **English** for immediate, human-readable reporting.